In [88]:
import numpy as np
import matplotlib.pyplot as plt

In [89]:
ROWS = 5    # Ajusta el número de filas a 5
COLS = 12   # Cols
S = (4, 0)  # Ajusta la posición inicial del agente
G = (4, 11) # Punto de llega

In [90]:
#clase Acantilado
class Cliff:
    def __init__(self):
        self.end = False
        self.pos = S     #Pos inicial del agente tupla(fila, columna)
        self.board = np.zeros([5, 12]) #creamos el tablero

        # Añade dos filas de acantilado marcadas como -1
        # creamos los obstaculos
        self.board[4, 1:11] = -1
        self.board[3, 2:10] = -1

    #calculamos la siguiente posición del agente según la acción proporcionada
    def nxtPosition(self, action):
        if action == "up":
            nxtPos = (self.pos[0] - 1, self.pos[1])
        elif action == "down":
            nxtPos = (self.pos[0] + 1, self.pos[1])
        elif action == "left":
            nxtPos = (self.pos[0], self.pos[1] - 1)
        else:
            nxtPos = (self.pos[0], self.pos[1] + 1)

        # Verifica la legitimidad, Actualiza la posición del agente, si la posición es válida dentro de los límites del tablero.
        if 0 <= nxtPos[0] < ROWS and 0 <= nxtPos[1] < COLS:
            self.pos = nxtPos

        #Verificamos la llegada al objetivo
        if self.pos == G:
            self.end = True
            print("✅ LLego a la meta")
        if self.board[self.pos] == -1:
            self.end = True
            print("💀 Cayo al acantilado")

        return self.pos

    #damos la recompensa al agente de acuerdo a su posicion
    def giveReward(self):
        if self.pos == G:
            return -1
        if self.board[self.pos] == 0:
            return -1
        return -100

  #  def show(self):
  #      for i in range(0, ROWS):
  #          print('-----------------------------------------------------')
  #          out = '| '
  #          for j in range(0, COLS):
  #              if self.board[i, j] == -1:
  #                  token = '*'
  #              if self.board[i, j] == 0:
  #                  token = '0'
  #              if (i, j) == self.pos:
  #                  token = 'S'
  #              if (i, j) == G:
  #                  token = 'G'
  #              out += token + ' | '
  #         print(out)
  #     print('-----------------------------------------------------')



In [91]:

class Agent:
    def __init__(self, exp_rate=0.3, lr=0.1, sarsa=True):
        self.cliff = Cliff()
        self.actions = ["up", "left", "right", "down"]
        self.states = []  # Registrar posición y acción de cada episodio,
        self.pos = S      # asignamos la posicion inicial
        self.exp_rate = exp_rate #tasa de exploracion
        self.lr = lr             #tasa de aprendizaje gmm
        self.sarsa = sarsa       #algoritmo / sarsa o q-lerning
        self.state_actions = {}  #almacenamos las estimaciones de valor para cada par estado-acción
        for i in range(ROWS):
            for j in range(COLS):
                self.state_actions[(i, j)] = {}
                for a in self.actions:
                    self.state_actions[(i, j)][a] = 0

    def chooseAction(self):
        # Épsilon-greedy
        mx_nxt_reward = -999 #Valor bajo para asegurar que cualquier recompensa sea mayor
        action = ""

        if np.random.uniform(0, 1) <= self.exp_rate:
            #Vemos si un nro aleatorio entre 0 y 1 es menor o igual a la tasa de exploración
            #elige una acción al azar de todas las posibles
            action = np.random.choice(self.actions)
        else:
            # Acción codiciosa
            for a in self.actions:
                #Recompensa estimada para cada acción en la posición actual
                current_position = self.pos
                nxt_reward = self.state_actions[current_position][a]

                #Si recompensa estimada con la mayor recompensa encontrada hasta ahora
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward

        return action

    def reset(self):
        self.states = []
        self.cliff = Cliff()
        self.pos = S

    def play(self, rounds=10):
        rewards = []
        for _ in range(rounds):
            while 1:
                curr_state = self.pos
                cur_reward = self.cliff.giveReward()
                action = self.chooseAction()

                # Próxima posición
                self.cliff.pos = self.cliff.nxtPosition(action)
                self.pos = self.cliff.pos

                self.states.append([curr_state, action, cur_reward])

                if self.cliff.end:
                    break

            # Juego terminado, actualizar estimaciones
            reward = self.cliff.giveReward()
            rewards.append(reward)

            # La recompensa de todas las acciones en el estado final es la misma
            for a in self.actions:
                self.state_actions[self.pos][a] = reward

            if self.sarsa:
                for s in reversed(self.states):     #estados almacenados en orden inverso
                    pos, action, r = s[0], s[1], s[2]                 #posicion, accion, recompensa en el estado act
                    current_value = self.state_actions[pos][action]   #Valor estimado  para la acción en la posición actual

                    #Recompensa acumulada en el episodio hasta el momento
                    reward = current_value + self.lr * (r + reward - current_value)

                    #Almacena la nueva estimación de valor redondeada
                    self.state_actions[pos][action] = round(reward, 3)
            else:
                for s in reversed(self.states):
                    pos, action, r = s[0], s[1], s[2]
                    current_value = self.state_actions[pos][action]

                    reward = current_value + self.lr * (r + reward - current_value)
                    self.state_actions[pos][action] = round(reward, 3)

                    #Actualizamos recompensa con valor máximo de todas las acciones posibles en la siguiente posición
                    reward = np.max(list(self.state_actions[pos].values()))  # Máximo

            self.reset()

In [92]:
def showRoute(states):
    board = np.zeros([5, 12])

    board[4, 1:11] = -1
    board[3, 2:10] = -1
    for i in range(0, ROWS):
        print('-------------------------------------------------')
        out = '| '
        for j in range(0, COLS):
            token = '🍀'
            if board[i, j] == -1:
                token = '🔥'
            if (i, j) in states:
                token = '👮'
            if (i, j) == G:
                token = '✅'
            out += token + ' | '
        print(out)
    print('-------------------------------------------------')


In [93]:
if __name__ == "__main__":
    print("============================= ")
    print(".::  Entrenamiento SARSA  ::. ")
    print("============================= ")
    ag = Agent(exp_rate=0.1, sarsa=True)
    ag.play(rounds = 500)

    # SARSA
    ag_op = Agent(exp_rate=0)
    ag_op.state_actions = ag.state_actions

    states = []
    while 1:
        curr_state = ag_op.pos
        action = ag_op.chooseAction()
        states.append(curr_state)
        print("Posición actual {}  | => {}".format(curr_state, action))

        # Próxima posición
        ag_op.cliff.pos = ag_op.cliff.nxtPosition(action)
        ag_op.pos = ag_op.cliff.pos

        if ag_op.cliff.end:
            break

    showRoute(states)



    print("===================================")
    print(".::  Entrenamiento Q-learning  ::. ")
    print("===================================")

    ag = Agent(exp_rate=0.1, sarsa=False)
    ag.play(rounds = 500)

    # Q-learning
    ag_op = Agent(exp_rate=0)
    ag_op.state_actions = ag.state_actions

    states = []
    while 1:
        curr_state = ag_op.pos
        action = ag_op.chooseAction()
        states.append(curr_state)
        print("Posición actual {}  | -> {}".format(curr_state, action))

        # Próxima posición
        ag_op.cliff.pos = ag_op.cliff.nxtPosition(action)
        ag_op.pos = ag_op.cliff.pos

        if ag_op.cliff.end:
            break

    showRoute(states)


.::  Entrenamiento SARSA  ::. 
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
✅ LLego a la meta
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo al acantilado
💀 Cayo